# 条件随机场 CRF

## 目录

- [概率无向图模型](#概率无向图模型)
- [条件随机场](#条件随机场)
    - [参数化形式](#线性链条件随机场的参数化形式)
    - [简化形式](#简化形式)
    - [矩阵形式](#矩阵形式)
- [三个问题](#三个问题)
    - [概率计算问题](#概率计算问题)
    - [学习方法](#学习方法)
    - [预测算法](#预测算法)
    

## 概率无向图模型

回顾一下之前讲解的概率无向图模型：https://applenob.github.io/graph_model.html

总结一下：

- **最大团**：无向图$G$中任何两个结点都有边连接的结点子集称为团（clique）。若团$C$不能在加入任何一个结点使其称为一个更大的团，则称$C$为图$G$的一个最大团。
- 概率无向图模型的**联合概率分**布可以表示成其最大团上的随机变量的函数的乘积形式。这也被称为概率无向图模型的**因子分解**。
- $P(Y) = \frac{1}{Z}\prod_C\psi_C(Y_C)$，其中，$Z$是规范化因子，$Z = \sum_Y\prod_C\psi_C(Y_C)$，$\psi_C(Y_C)$称为**势函数**，要求势函数是严格正的（因为涉及到累乘）。

## 条件随机场

条件随机场（Conditional Random Field， CRF）也是一种无向图模型。它是在给定随机变量$X$的条件下，随机变量$Y$的马尔科夫随机场。

我们常用的是**线性链条件随机场**，多用于序列标注等问题。形式化定义：设$X=(X_1, X_2, ..., X_n)$，$Y=(Y_1, Y_2, ..., Y_n)$均为线性链表示的随机变量序列，若给在定随机变量序列$X$的条件下，随机变量序列$Y$的条件概率分布$P(Y|X)$构成条件随机场，即满足**马尔科夫性**：$P(Y_i|X, Y_1,...,Y_{i-1}, Y_{i+1}, ..., Y_n) = P(Y_i|X, Y_{i-1}, Y_{i+1})\;\;i=1,2, ..., n$（**这个式子是核心，充分理解这个式子和下面的图片**），则称$P(Y|X)$是线性链条件随机场。

![](https://github.com/applenob/machine_learning_basic/raw/master/res/linear_crf.png)

### 参数化形式

$$P(y|x) = \frac{1}{Z(x)}exp(\sum_{i,k}\lambda_kt_k(y_{i-1}, y_i, x, i)+\sum_{i,l}\mu_ls_l(y_i, x, i))$$

其中，$Z(x) = \sum_yexp(\sum_{i,k}\lambda_kt_k(y_{i-1}, y_i, x, i)+\sum_{i,l}\mu_ls_l(y_i, x, i))$，$t_k$是转移(transform)特征函数，依赖于当前和前一个位置，$s_l$是状态(state)特征函数，依赖于当前位置，$\lambda_k$和$\mu_l$是对应的权值。

从模型的参数化形式可以看出，线性链条件随机场也是对数线性模型。

### 简化形式

所谓简化形式即，将局部特征特征统一成一个全局特征函数。

设有$K_1$个转移特征，有$K_2$个状态特征，$K=K_1+K_2$。

$$f_k(y_{i-1}, y_i, x, i) = \left\{\begin{matrix}t_k(y_{i-1}, y_i, x, i),\;\;k=1,2,...,K_1\\ s_l(y_i, x, i),\;\; k=K_1+l,\; l=1,2,..,K_2\end{matrix}\right.$$

对所有在位置$i$的特征求和：

$$f_k(y,x) = \sum^n_{i=1}f_k(y_{i-1}, y_i, x, i), \;\; k=1,2,...,K$$

用$w_k$表示特征$f_k(y,x)$的权值，即：$w_k = \left\{\begin{matrix}\lambda_k,\;\;k=1,2,...,K_1\\ \mu_l,\;\;k=K_1+l;l=1,2,...,K_2\end{matrix}\right.$

于是条件随机场又可以表示成：

$$P(y|x) = \frac{1}{Z(x)}exp\sum^K_{k=1}w_kf_k(y,x)\\Z(x)=\sum_y exp\sum^K_{k=1}w_kf_k(y,x)$$

如果用$w$表示权值向量，即$w=(w_1, ..., w_K)^T$，用$F(y,x)$表示全局特征向量，即$F(y,x) = (f_1(y,x),f_2(y,x),...,f_K(y,x))^T$，则条件随机场可以携程向量$w$和$F(y,x)$的内积的形式：
$$P_w(y|x) = \frac{exp(w\cdot F(y,x))}{Z_w(x)}\\Z(x)=\sum_yexp(w\cdot F(y,x))$$

### 矩阵形式

引入特殊的起点和终点状态标记$y_0=start$，$y_{n+1}=stop$。

对于观测序列$x$的每一个位置$i=1,2,...,n+1$定义$n+1$个$m$阶方阵（$m$是标记$y_i$取值的个数）。
- $M_i(x) = [M_i(y_{i-1}, y_i| x)]$
- $M_i(y_{i-1}, y_i| x) = exp(W_i(y_{i-1}, y_i| x))$
- $W_i(y_{i-1}, y_i| x) = \sum_{k=1}^Kw_kf_k(y_i,y_i,x,i)$

条件随机场的矩阵形式：

$$P_w(y|x) = \frac{1}{Z_w(x)}\prod^{n+1}_{i=1}w_kf_k(y_{i-1},y_i,x,i)\\Z_w(x)=(M_1(x)M_2(x)...M_{n+1}(x))_{start, stop}$$

## 三个问题

类似于隐马尔科夫模型（HMM），CRF也有典型的三个问题。对比二者在这三个问题的解决方法的不同，可以更深入理解这两个模型。

- 1.**概率计算问题**：给定条件随机场$P(Y|X)$，输入序列$x$和输出序列$y$，计算条件概率$P(Y_i=y_i|x)$和$P(Y_{i-1}=y_{i-1}, Y_i=y_i|x)$和相应的数学期望。
- 2.**学习问题**：给定训练数据集，估计条件随机场模型参数，即用**极大似然法**的方法估计参数。
- 3.**预测问题**：给定条件随机场$P(Y|X)$和输入序列（观测序列）$x$，求条件概率最大的输出序列（标记序列）$y^*$。

### 概率计算问题

给定条件随机场$P(Y|X)$，输入序列$x$和输出序列$y$，计算条件概率$P(Y_i=y_i|x)$和$P(Y_{i-1}=y_{i-1}, Y_i=y_i|x)$。

类似于HMM，引入前向-后向向量。

### 学习方法

给定训练数据集，估计条件随机场模型参数，即用**极大似然法**的方法估计参数。

### 预测算法

给定条件随机场$P(Y|X)$和输入序列（观测序列）$x$，求条件概率最大的输出序列（标记序列）$y^*$。